In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01/
!ls

[Errno 2] No such file or directory: '/content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01/'
/home/jovyan/work/workplace/AIFFEL_quest_rs/Exploration/Quest01/etc_teat
Gpt-2.ipynb  Transformer_Translator.ipynb  Untitled0.ipynb  Untitled1.ipynb


In [2]:
!pip install Korpora

In [3]:
# 네이버 영화 리뷰 데이터셋(NSMC) 로드 및 샘플링
import numpy as np # 넘파이 라이브러리 임포트
import pandas as pd # 판다스 라이브러리 임포트
from Korpora import Korpora # Korpora 라이브러리에서 Korpora 클래스 임포트

corpus = Korpora.load("nsmc") # 'nsmc' 데이터셋을 불러와 corpus 변수에 저장
# Korpora.load() 함수는 'nsmc' 데이터셋을 불러오는 역할을 합니다.

df = pd.DataFrame(corpus.test).sample(20000, random_state=42) # 데이터프레임 생성 및 샘플링
# corpus.test는 테스트 데이터셋을 의미합니다.
# pd.DataFrame()을 이용해 테스트 데이터를 데이터프레임으로 변환합니다.
# .sample(20000, random_state=42)를 통해 전체 테스트 데이터 중 20,000개를 무작위로 추출하되,
# random_state=42를 설정하여 매번 동일한 결과를 얻도록 합니다.


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /home/jovyan/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /home/jovyan/K

In [4]:
!pip install tabulate

In [5]:
# 데이터프레임을 훈련, 검증, 테스트 데이터셋으로 분할
# df.sample(frac=1, random_state=42) : 데이터프레임 'df'의 모든 행을 무작위로 섞습니다.
#                                      frac=1은 전체 데이터를 의미하고, random_state는 재현성을 위해 고정합니다.
# np.split(...) : 섞인 데이터를 지정된 비율로 나눕니다.
# [int(0.6 * len(df)), int(0.8 * len(df))] : 분할 기준이 되는 인덱스를 계산합니다.
#                                            전체 길이의 60% 지점과 80% 지점을 기준으로 데이터를 3개로 나눕니다.
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))])

# print(train.head(5).to_markdown()) : 훈련 데이터셋의 상위 5개를 마크다운 형식으로 출력합니다.
# print(f"Training Data Size : {len(train)}") : 훈련 데이터셋의 크기를 출력합니다.
# print(f"Validation Data Size : {len(valid)}") : 검증 데이터셋의 크기를 출력합니다.
# print(f"Testing Data Size : {len(test)}") : 테스트 데이터셋의 크기를 출력합니다.
print(train.head(5).to_markdown())
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

|       | text                                                     |   label |
|------:|:---------------------------------------------------------|--------:|
| 26891 | 역시 코믹액션은 성룡, 홍금보, 원표 삼인방이 최고지!!     |       1 |
| 25024 | 점수 후하게 줘야것네 별 반개~                            |       0 |
| 11666 | 오랜만에 느낄수 있는 [감독] 구타욕구.                    |       0 |
| 40303 | 본지는 좀 됬지만 극장서 돈주고 본게 아직까지 아까운 영화 |       0 |
| 18010 | 징키스칸이란 소재를 가지고 이것밖에 못만드냐             |       0 |
Training Data Size : 12000
Validation Data Size : 4000
Testing Data Size : 4000


/opt/conda/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
# BERT 모델 학습을 위한 데이터 전처리 및 데이터로더 생성
import torch # 파이토치 라이브러리 임포트
from transformers import BertTokenizer # Hugging Face의 transformers 라이브러리에서 BertTokenizer를 임포트합니다.
from torch.utils.data import TensorDataset, DataLoader # 파이토치 데이터 유틸리티를 임포트합니다.
from torch.utils.data import RandomSampler, SequentialSampler # 데이터 샘플러를 임포트합니다.

# 데이터셋을 만드는 함수를 정의합니다.
def make_dataset(data, tokenizer, device):
    # 입력 데이터를 토크나이징합니다.
    tokenized = tokenizer(
        text=data.text.tolist(), # 데이터프레임의 'text' 컬럼을 리스트로 변환하여 입력으로 사용합니다.
        padding="longest", # 모든 시퀀스의 길이를 가장 긴 시퀀스에 맞춰 패딩합니다.
        truncation=True, # 시퀀스 길이가 최대 길이를 초과하면 잘라냅니다.
        return_tensors="pt" # 파이토치 텐서 형태로 반환합니다.
    )
    # 토큰화된 데이터에서 input_ids(토큰 인덱스)를 추출하여 device로 이동시킵니다.
    input_ids = tokenized["input_ids"].to(device)
    # 토큰화된 데이터에서 attention_mask(어텐션 마스크)를 추출하여 device로 이동시킵니다.
    attention_mask = tokenized["attention_mask"].to(device)
    # 데이터프레임의 'label' 컬럼을 파이토치 텐서로 변환하고 device로 이동시킵니다.
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    # input_ids, attention_mask, labels를 묶어 TensorDataset 객체를 반환합니다.
    return TensorDataset(input_ids, attention_mask, labels)

# 데이터로더를 생성하는 함수를 정의합니다.
def get_datalodader(dataset, sampler, batch_size):
    # 데이터셋에 적용할 샘플러(Random 또는 Sequential)를 생성합니다.
    data_sampler = sampler(dataset)
    # 데이터셋, 샘플러, 배치 사이즈를 이용하여 DataLoader를 생성합니다.
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    # 생성된 데이터로더를 반환합니다.
    return dataloader

# 하이퍼파라미터를 설정합니다.
epochs = 5
batch_size = 32
# GPU 사용 가능 여부를 확인하여 device를 설정합니다.
device = "cuda" if torch.cuda.is_available() else "cpu"

# BERT 토크나이저를 로드합니다.
# pretrained_model_name_or_path: 사전 학습된 모델의 이름을 지정합니다. (다국어 BERT 모델)
# do_lower_case: 소문자화 여부를 결정합니다. (False는 소문자화하지 않음)
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    do_lower_case=False)

# make_dataset 함수를 사용하여 훈련, 검증, 테스트 데이터셋을 생성합니다.
train_dataset = make_dataset(train, tokenizer, device)
valid_dataset = make_dataset(valid, tokenizer, device)
test_dataset = make_dataset(test, tokenizer, device)

# get_datalodader 함수를 사용하여 각 데이터셋에 대한 데이터로더를 생성합니다.
# 훈련 데이터로더는 RandomSampler를 사용하여 데이터를 무작위로 섞습니다.
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)
# 검증 및 테스트 데이터로더는 SequentialSampler를 사용하여 데이터를 순차적으로 사용합니다.
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

# 훈련 데이터셋의 첫 번째 요소를 출력합니다.
print(train_dataset[0])

(tensor([   101,  58466,   9812, 118956, 119122,  59095,  10892,   9434, 118888,
           117,   9992,  40032,  30005,    117,   9612,  37824,   9410,  12030,
         42337,  10739,  83491,  12508,    106,    106,    102,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,

In [7]:
!pip install transformers

In [8]:
# import torch

# # GPU 사용 가능 여부를 확인하여 device 변수를 정의합니다.
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # 이제 device 변수를 사용할 수 있습니다.
# model = BertForSequenceClassification.from_pretrained(
#     pretrained_model_name_or_path="bert-base-multilingual-cased",
#     num_labels=2).to(device)

NameError: name 'BertForSequenceClassification' is not defined

In [9]:
# BERT 모델 및 옵티마이저 설정
from torch import optim # 파이토치에서 옵티마이저 모듈을 임포트합니다.
from transformers import BertForSequenceClassification # Hugging Face의 transformers 라이브러리에서 BertForSequenceClassification 클래스를 임포트합니다.

# BertForSequenceClassification 모델을 불러옵니다.
model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased", # 다국어 BERT 모델을 사용합니다.
    num_labels=2).to(device) # 분류할 라벨의 개수를 2개(긍정/부정)로 설정하고, 모델을 지정된 device(CPU 또는 GPU)로 이동시킵니다.
    # .to(device)는 모델을 GPU와 같은 가속기로 보내 연산을 빠르게 합니다.

# AdamW 옵티마이저를 정의합니다.
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# model.parameters(): 모델의 모든 학습 가능한 파라미터를 옵티마이저에 전달합니다.
# lr=1e-5: 학습률(learning rate)을 0.00001로 설정합니다.
# eps=1e-8: 부동 소수점 연산에서 0으로 나누는 것을 방지하기 위한 작은 상수(epsilon)를 설정합니다.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# BERT 모델의 계층 구조 출력
# model.named_children(): 모델의 직계 자식 모듈들을 (이름, 모듈) 쌍으로 반환합니다.
for main_name, main_module in model.named_children():
    print(main_name) # 'bert'와 'dropout', 'classifier' 같은 최상위 모듈의 이름을 출력합니다.
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name) # 최상위 모듈의 하위 모듈 이름을 출력합니다. (예: 'bert' 모듈 아래의 'embeddings', 'encoder', 'pooler')
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name) # 하위 모듈의 하위 모듈 이름을 출력합니다. (예: 'encoder' 모듈 아래의 'layer')
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name) # 가장 깊은 계층의 모듈 이름을 출력합니다. (예: 'layer' 모듈 아래의 '0'부터 '11'까지)

bert
└ embeddings
│  └ word_embeddings
│  └ position_embeddings
│  └ token_type_embeddings
│  └ LayerNorm
│  └ dropout
└ encoder
│  └ layer
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  │  └ 6
│  │  └ 7
│  │  └ 8
│  │  └ 9
│  │  └ 10
│  │  └ 11
└ pooler
│  └ dense
│  └ activation
dropout
classifier


In [11]:
# BERT 모델 훈련, 검증 및 가중치 저장
# BERT 모델 훈련, 검증 및 가중치 저장
import numpy as np
import torch
from torch import nn
from tqdm.auto import tqdm # tqdm 라이브러리 임포트
from transformers import BertTokenizer


# 하이퍼파라미터와 디바이스 설정
epochs = 5
batch_size = 32
device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer 정의
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    do_lower_case=False)

# 데이터셋 및 데이터로더 생성
train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)
valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)
test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)


# 정확도를 계산하는 함수입니다.
def calc_accuracy(preds, labels):
    # 예측값(preds)의 가장 높은 확률을 가진 클래스의 인덱스를 가져와 1차원 배열로 만듭니다.
    pred_flat = np.argmax(preds, axis=1).flatten()
    # 정답 라벨(labels)을 1차원 배열로 만듭니다.
    labels_flat = labels.flatten()
    # 예측값과 정답이 일치하는 비율을 계산하여 반환합니다.
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 모델을 훈련시키는 함수입니다.
def train(model, optimizer, dataloader):
    model.train() # 모델을 훈련 모드로 설정합니다.
    train_loss = 0.0 # 훈련 손실을 초기화합니다.

    # 데이터로더로부터 배치 단위로 데이터를 가져옵니다.
    for input_ids, attention_mask, labels in tqdm(dataloader, desc="Training"):
        # 모델에 데이터를 입력하여 출력(outputs)을 얻습니다.
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss # 출력에서 손실(loss) 값을 추출합니다.
        train_loss += loss.item() # 손실 값을 누적합니다.

        optimizer.zero_grad() # 옵티마이저의 기울기를 초기화합니다.
        loss.backward() # 손실에 대한 역전파를 수행하여 기울기를 계산합니다.
        optimizer.step() # 옵티마이저를 사용하여 모델 파라미터를 업데이트합니다.

    train_loss = train_loss / len(dataloader) # 배치당 평균 손실을 계산합니다.
    return train_loss # 평균 훈련 손실을 반환합니다.

# 모델을 평가하는 함수입니다.
def evaluation(model, dataloader):
    with torch.no_grad(): # 기울기 계산을 비활성화하여 메모리를 절약하고 연산 속도를 높입니다.
        model.eval() # 모델을 평가 모드로 설정합니다.
        criterion = nn.CrossEntropyLoss() # 손실 함수로 교차 엔트로피 손실을 사용합니다.
        val_loss, val_accuracy = 0.0, 0.0 # 검증 손실과 정확도를 초기화합니다.

        # 데이터로더로부터 배치 단위로 데이터를 가져옵니다.
        for input_ids, attention_mask, labels in tqdm(dataloader, desc="Evaluation"):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # 모델에 데이터를 입력합니다.
            logits = outputs.logits # 출력에서 로짓(logits) 값을 추출합니다.

            loss = criterion(logits, labels) # 교차 엔트로피 손실을 계산합니다.
            logits = logits.detach().cpu().numpy() # 로짓을 CPU로 이동 후 넘파이 배열로 변환합니다.
            label_ids = labels.to("cpu").numpy() # 라벨을 CPU로 이동 후 넘파이 배열로 변환합니다.
            accuracy = calc_accuracy(logits, label_ids) # 정확도를 계산합니다.

            val_loss += loss.item() # 손실 값을 누적합니다.
            val_accuracy += accuracy # 정확도를 누적합니다.

    val_loss = val_loss/len(dataloader) # 배치당 평균 손실을 계산합니다.
    val_accuracy = val_accuracy/len(dataloader) # 배치당 평균 정확도를 계산합니다.
    return val_loss, val_accuracy # 평균 손실과 정확도를 반환합니다.



# 훈련 루프를 시작합니다.
best_loss = 10000 # 최저 손실을 저장하기 위한 변수를 큰 값으로 초기화합니다.
for epoch in range(epochs): # 지정된 에포크(epochs) 수만큼 반복합니다.
    print(f"--- Epoch {epoch + 1}/{epochs} ---") # 에포크 진행 상황을 시각적으로 표시
    train_loss = train(model, optimizer, train_dataloader) # 훈련 함수를 호출합니다.
    val_loss, val_accuracy = evaluation(model, valid_dataloader) # 평가 함수를 호출합니다.
    # 에포크별 훈련 손실, 검증 손실, 검증 정확도를 출력합니다.
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Accuracy {val_accuracy:.4f}")

    # 현재 검증 손실이 기존의 최저 손실보다 낮으면 모델을 저장합니다.
    if val_loss < best_loss:
        best_loss = val_loss # 최저 손실을 업데이트합니다.
        torch.save(model.state_dict(), "../models/BertForSequenceClassification.pt") # 모델의 가중치를 저장합니다.
        print("Saved the model weights") # 모델이 저장되었음을 알립니다.

--- Epoch 1/5 ---


Training:   0%|          | 0/375 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 1: Train Loss: 0.5755 Val Loss: 0.4578 Val Accuracy 0.7870
Saved the model weights
--- Epoch 2/5 ---


Training:   0%|          | 0/375 [00:00<?, ?it/s]

...

Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 4: Train Loss: 0.2662 Val Loss: 0.4673 Val Accuracy 0.8153
--- Epoch 5/5 ---


Training:   0%|          | 0/375 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

Epoch 5: Train Loss: 0.2136 Val Loss: 0.4797 Val Accuracy 0.8150


In [12]:
# 저장된 모델 가중치를 불러와 테스트 데이터셋으로 성능 평가
# BertForSequenceClassification 모델을 불러옵니다.
model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased", # 다국어 BERT 모델을 사용합니다.
    num_labels=2).to(device) # 분류할 라벨의 개수를 2개로 설정하고, 모델을 지정된 device로 이동시킵니다.
    # .to(device)는 모델을 GPU와 같은 가속기로 보내 연산을 빠르게 합니다.

# 저장된 모델의 가중치(state_dict)를 불러와 현재 모델에 적용합니다.
# torch.load() 함수를 이용해 파일 경로에 있는 가중치 파일을 불러옵니다.
# model.load_state_dict() 메서드를 사용하여 불러온 가중치를 모델에 로드합니다.
model.load_state_dict(torch.load("../models/BertForSequenceClassification.pt"))

# evaluation 함수를 사용하여 모델의 테스트 손실과 정확도를 계산합니다.
test_loss, test_accuracy = evaluation(model, test_dataloader)

# 계산된 테스트 손실과 테스트 정확도를 출력합니다.
print(f"Test Loss : {test_loss:.4f}")
print(f"Test Accuracy : {test_accuracy:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

Test Loss : 0.4030
Test Accuracy : 0.8193
